**TEMA**

PREDICCIÓN DE PRECIOS EN EL MERCADO INMOBILIARIO DE MADRID.


**Suponemos el siguiente problema de negocio:**

Una empresa inmobiliaria de Madrid nos pide que hagamos un modelo para poder generar precios de compraventa de inmuebles lo más ajustados a la realidad posibles.
  
La  empresa considera que esos precios son muy útiles para el negocio, por motivos como los siguientes:  
- Para ofrecer rápidamente a un posible venededor una estimación del precio al que podrían poner sus inmuebles en venta. Eso ayuda a captar clientes.
- Para orientar a los compradores sobre los precios de inmuebles que reunan características que ellos demandan.
- Para la propia empresa, para conocer cómo está el mercado y poder destinar recursos a las operaciones  a priori más rentables.
- Pare tener una herramienta que se pueda actualizar con nueva información y así adaptarse a la rápida evolución de este sector.

**Obtención de los datos**

Para ello se va a usar una base de datos obtenida a traves de KAGGLE. 
Se trata de una agrupacion de información de portales inmobiliarios de inmuebles situados en MADRID con más de 20.000 registros, lo que debería darle mucha solidez.  

 Se han obtenido datos de la web https://www.kaggle.com/mirbektoktogaraev/madrid-real-estate-market. Kaggle (propiedad de Google) es una plataforma en línea que  permite a los usuarios encontrar y publicar conjuntos de datos, explorar y crear modelos en un entorno basado en la web. Es una web pública con datos de anfitriones que trabajan con la plataforma.

En la base de datos hay muchas variables, tanto descriptivas (con texto) como categóricas, numéricas y booleanas.  
Veremos si se pueden eliminar todas las que no tengan datos suficientes o no aporten casi información o esta sea redundante respecto a otra variable.


**Objetivos:**

- Ver qué factores están condicionando el precio y si alguno tiene relevancia especial.
- Obtener el modelo más sencillo, rápido y flexible posible.
- Que el modelo sea fácil de mantener, reentrenar y de adaptar a datos de otras ciudades.

**La hipótesis de partida:**  

Pensamos que:
- Hay varibles que por su natureza suelen tener mucha relación con el precio final, como pueden ser los metros cuadrados, número de habitaciones, localización ,etc.

Otros supuestos menores menores son:  

- Hay otros datos que son más bien burocráticos y que no sabemos qué relación con el precio final tendrán como la calificación energética.
- Hay información que solo se porporciona en algunos casos, supongamos que es también útil para nuestro modelo y ya veremos como si tiene jardín, es exterior, tiene calefacción etc.
- Hay información contenida en algunas variables que también se puede encontrar en otras variables como identiicadores de localización, barrio, distrito, calle, etc. Posiblemente pueda prescindirse de alguno para que sea más fácil y barato tratar los datos.
- Hay variables con información muy relacionada entre sí y con posiblemente el mismo efecto en el precio de venta, como puedan ser los metros cuadrados construídos, el número de habitaciones, etc. Posiblemente se pueda prescindir de alguna también para agilizar el procesado.
- Cuantas menos variables necesitemos para predecir más fácil  y barato resultará para la empresa usar y mantener el modelo.

**Preparación de funciones auxiliares**

Se han programado y puesto en un script una serie de funciones que puden ser llamadas de forma recurrente desde el notebook principal para ver aspectos como la descripción de los datos, las cardinalidades, los *missings*, etc y que evitan tener que escribir mucho código repetitivo.

**Herramientas de visualización**

Se prepara un scrip con herramientas de visualización que son muy útiles.

**Visualización inicial**

Nos descagamos las librerias de python que podemos necesitar.
Tras la carga de los datos, se muestran en un dataframe y se ve la tipología asignada por Python por defecto.

**Separación entre train y test**

Inmediatamente, antes de hacer nada, se separan los datos en un 80% para entrenar (train) y un 20% para probar (test).

**Análisis exploratorio de datos (EDA)**

Definimos como *target* el precio de compra,  como es natural, porque es lo que intentamos predecir.
Se hace un visualizado masivo de todas las distribuciones y las correlaciones de todas las variables.

Como el  preprocesado lo haremos con *pipelines* se van analizando las variables y han ido creando cuatro listas:

- **Variables categóricas**: el tipo de inmueble, el distrito y el certificado energético.
- **Variables numércias**: comprende los metros cuadrados, el número  de habitaciones, el número de baños y los precios por metro cuadrado según las zonas.
- **Variables booleanas**: algunas aparecían como *object* pero eran boolenas encubiertas.  
- **Variables a eliminar**: el resto.



**Preprocesado:**

Se generan pipelines para las transformaciones de las variables de la siguiente forma:

- Variables categóricas: vamos a preparar un pipeline con un one-hot-encoder para desplegar cada categoría de forma individual en cada variable.
- Variables numéricas: como hemos visto que las distribuciones son algo irregulares, para facilitar la regresión vamos a aplicar un *StandardScaler* y un logaritmo.
- Variables a eliminar: generaremos un pipeline con la orden *drop* para eliminarlos.
- Variables booleanas: como ya están tratadas de antemano, no se van a tocar, el pipeline no las transformará.

Para ver las columnas que nos van quedando hemos ido creando una lista llamada "Columns Remaining" para ver las que nos quedan y observamos que tras aplicar el preprocesado tendríamos 23 columnas, que tras el preprocesado, por efecto del *one-hot-encoder* pasarían a ser 53. 
Por ahora no las reducimos hasta ver los tiempos de entrenamiento de los modelos.


**Elección del modelo**

Vamos a seleccinar cuatro modelos de regresión. Los hemos importado de las librerías *scikit-learn* y serán:  
- *LinearRegression*: con este modelo haremos una regresión linear multivariante. Este modelo es sencillo y rápido.
- *RandomForestRegressor*: también es sencillo y rápido.
- *XGBoostRegressor*: capta relaciones no lineales. Es potente para usar con muchos datos y de cierta complejidad. Aquí al haber tratado tanto los datos puede que no sea necesario, pero si en el futuro la base de datos se hace más complicada podría usarse. Además suele prevenenir el sobreajuste. Lo probamos.
- *LighGBMRegressor*: captura también relaciones no lineasles, escala muy bien para bases de datos grandes y maneja bien las categóricas. Si no pudiéramos tratar las categóricas nos ayudaría, aunque las hemos tratado. Lo probamos.

**Entrenamiento**

Creamos el pipeline de preprocesado como hemos indicado anteriormente y se deja para incluirlo en el entrenamiento.

Tran instanciar los cuatro modelos con **Pipelines** hacemos un *baseline* con **Cross Validation** para tomar como referencia las siguientes métricas: 

    - R² (Coeficiente de Determinación): cuanto más alto mejor, más queda explicada la variable dependiente por las independientes.
    - MSE(Errorr Cuadrático Medio): cuanto más bajo mejor, los errores serán más pequeños. 
    - MAE (Error Absoluto Medio): cuanto más bajo mejor, los errores serán más pequeños.
    - MEA (Mediana del Error Abosluto): cuanto más bajo mejor, los errores serán más pequeños.

**Resultados**

Sorprendentemente los cuatro modelos predicen muy bien.
El mejor es la ***Regresión lineal*** con un R2 de prácticamente 1. Resulta extraño, los datos podrían haberse contaminado y estar incluyendo el valor a predecir dentro del dataset de entrenamiento. 

Probamos a eliminar en un notebook aparte las dos variables que más correlación tenían con el target, metros cuadrados y número de baños y el R2 pasa a ser 0.80.  
Por ello no nos atrevemos a descartar el modelo y seguiremos adelante con él.
Más adelante habría que hacer más pruebas para ver si ha pasado algo con los datos.

**Prueba contra el test**

Entrenamos de nuevo el *Regresor Lineal*, lo probamos contra el set de prueba (*test_set*) y parece que el R2 ya baja a un 0.8, lo que es una métrica  más normal.
Extraremos los coeficientes y los ordenamos por peso de mayor a menor del regresor y ya podemos extraer conclusiones.

**Guardado**

Guardamos el modelo entrenado con formato .pkl para poderlo usar en comprobaciones posteriores o si fuera el caso dese una API en una web.

**CONCLUSIONES**

Los precios es la variable a determinar.  
No hay sorpresas en los determinantes del precio de venta de un inmueble en Madrid.  
Lo más importante es la localización y el tipo de vivienda.  
Luego los metros cuadrados y número de estancias/baños juegan también un papel decisivo.  
Hasta aquí nada nuevo, lo importante es que nuestro modelo una vez revisado podría determinar con bastante exactitud el precio de venta aproximado de un inmueble tras darle una serie de características.  
Y lo haría de forma extremadamente rápida.  
Cualquiera de los cuatro modelos manejados dan buenos resultados y todos se pueden adaptar a nuevas bases de datos de otras ciudades o reentrenar con nuevos datos según avance el tiempo y se produzcan más transacciones.










[def]: attachment:image.png